In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import skimage.io as skio
import matplotlib.pyplot as plt
import numpy as np
import porespy as ps
import os
import skimage
import PIL

# Step 1
### This code is the first step after manual segmentation in Avizo. We need to rescale our images before giving them to the StyleGAN2ADA because it was designed to work with 0-255 range. This way, for each segmentatied phase, we give a corresponding value in 0-255 range. Note that as input, we take .tif files, and as output, we provide .png files

In [21]:
from PIL import Image
import os

def process_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get the list of files in the input folder
    file_list = os.listdir(input_folder)

    for filename in file_list:
        if filename.endswith('.tif'):
            # Open the image
            image_path = os.path.join(input_folder, filename)
            image = Image.open(image_path)

            # Create a new image with modified pixel values
            modified_image = Image.new('L', image.size)  # 'L' mode for grayscale images

            # Iterate over each pixel and modify its value
            width, height = image.size
            for x in range(width):
                for y in range(height):
                    pixel_value = image.getpixel((x, y))

                    if pixel_value == 1:
                        modified_image.putpixel((x, y), 0)
                    elif pixel_value == 2:
                        modified_image.putpixel((x, y), 85)
                    elif pixel_value == 3:
                        modified_image.putpixel((x, y), 170)
                    elif pixel_value == 4:
                        modified_image.putpixel((x, y), 255)

            # Save the modified image in the output folder
            output_filename = os.path.splitext(filename)[0] + '.png'
            output_path = os.path.join(output_folder, output_filename)
            modified_image.save(output_path)

            # Close the images
            image.close()
            modified_image.close()

    print("Image processing completed.")

# Specify the input and output folders
input_folder = r'C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Segmented'
output_folder = r'C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Segm_scale'

# Process the images
process_images(input_folder, output_folder)

Image processing completed.


# Step 2
## In this step 2, we expand the training dataset. The file below can take .png files in the folder, extract subimages with pre-set overlap, and save them in the separate folder. Here the images with size 1024x1024 are generated. We use the images from the Step 1. After step 2, the images are ready for the StyleGAN2ADA. Archive them in .zip and don't forget the json configuration file. Give the correct name

In [38]:
import os
from PIL import Image

def extract_sub_images(input_folder, output_folder, overlap_percentage):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all .png files in the input folder
    png_files = [file for file in os.listdir(input_folder) if file.endswith('.png')]

    c = 0
    for png_file in png_files:
        # Load the .png image
        png_path = os.path.join(input_folder, png_file)
        image = Image.open(png_path)

        # Calculate the overlap amount based on the overlap percentage
        overlap_amount = int((1024 * overlap_percentage) / 100)

        # Calculate the number of sub-images in each dimension with the overlap
        num_sub_images_x = (image.width - overlap_amount) // (1024 - overlap_amount)
        num_sub_images_y = (image.height - overlap_amount) // (1024 - overlap_amount)

        for i in range(num_sub_images_x):
            for j in range(num_sub_images_y):
                # Calculate the starting coordinates of the sub-image
                x_start = i * (1024 - overlap_amount)
                y_start = j * (1024 - overlap_amount)

                # Extract the sub-image
                sub_image = image.crop((x_start, y_start, x_start + 1024, y_start + 1024))

                # Save the sub-image as a .png file with a six-digit name
                sub_image_name = f"img_{c}.png"
                output_path = os.path.join(output_folder, sub_image_name)
                sub_image.save(output_path)
                c += 1

        print(f"Sub-images extracted from {png_file} and saved in {output_folder}.")

# Example usage
input_folder = r"C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Segm_scale"
output_folder = r"C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\00000"
overlap_percentage = 80  # Set the desired overlap percentage
extract_sub_images(input_folder, output_folder, overlap_percentage)

Sub-images extracted from 1.png and saved in C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Simple_1024.
Sub-images extracted from 10.png and saved in C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Simple_1024.
Sub-images extracted from 11.png and saved in C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Simple_1024.
Sub-images extracted from 12.png and saved in C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Simple_1024.
Sub-images extracted from 13.png and saved in C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Simple_1024.
Sub-images extracted from 14.png and saved in C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Simple_1024.
Sub-images extracted from 2.png and saved in C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Simple_1024.
Sub-images extracted from 3.png and saved in C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Simple_1024.
Sub-images extracted from 4.png and saved in C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Simple_1024.
Sub-i

# Step 3
## After the StyleGAN2ADA is trained and working well, it generates images with values in some range. This piece of code takes these artificially generated images and put them back into a single-value values for each pixel. For the Super-Resolution code, we should choose 0, 1, 2, 3, etc

In [48]:
from PIL import Image
import os

def process_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get the list of files in the input folder
    file_list = os.listdir(input_folder)

    for filename in file_list:
        if filename.endswith('.png'):
            # Open the image
            image_path = os.path.join(input_folder, filename)
            image = Image.open(image_path)

            # Create a new image with modified pixel values
            modified_image = Image.new('L', image.size)  # 'L' mode for grayscale images

            # Iterate over each pixel and modify its value
            width, height = image.size
            for x in range(width):
                for y in range(height):
                    pixel_value = image.getpixel((x, y))

                    if pixel_value > -30 and pixel_value < 45:
                        modified_image.putpixel((x, y), 0)
                    elif pixel_value > 45 and pixel_value < 130:
                        modified_image.putpixel((x, y), 85)
                    elif pixel_value > 130 and pixel_value < 210:
                        modified_image.putpixel((x, y), 170)
                    elif pixel_value > 210 and pixel_value < 275:
                        modified_image.putpixel((x, y), 255)

            # Save the modified image in the output folder
            output_filename = os.path.splitext(filename)[0] + '.png'
            output_path = os.path.join(output_folder, output_filename)
            modified_image.save(output_path)

            # Close the images
            image.close()
            modified_image.close()

    print("Image processing completed.")

# Specify the input and output folders
input_folder = r'C:\Users\UGOLKOEA\Desktop\test\try_here'
output_folder = r'C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Art_generated'

# Process the images
process_images(input_folder, output_folder)

Image processing completed.


## This piece of code was generated for the purpose of Avizo DL segmentation. To run their code, we need to stack our 2D images in the first stack, and to stack our segmented labels into the second stack. This code goes into the folder with our 2D images and stack them into the single 3D. Then, we can give this stack into the Avizo ML segmentation algorithm

In [1]:
import os
import numpy as np
from PIL import Image
import tifffile as tiff

# Path to the folder containing the PNG images
folder_path = r"C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20\Art_generated"

# Get a list of all PNG files in the folder
png_files = [f for f in os.listdir(folder_path) if f.endswith(".png")]

# Sort the files alphabetically
png_files.sort()

# Load the first image to get the dimensions
first_image_path = os.path.join(folder_path, png_files[0])
first_image = Image.open(first_image_path)
width, height = first_image.size

# Create an empty 3D volume
volume = np.zeros((len(png_files), height, width), dtype=np.uint8)

# Iterate over the PNG files and stack them in the volume
for i, png_file in enumerate(png_files):
    image_path = os.path.join(folder_path, png_file)
    image = Image.open(image_path)
    volume[i, :, :] = np.array(image)

# Save the volume as a TIFF file
output_directory = r"C:\Users\UGOLKOEA\Desktop\PhD_Thesis\Images\Good\20"
output_path = os.path.join(output_directory, "output.tiff")
tiff.imsave(output_path, volume)